# 02. Random Walk Model

**Mô hình:** Random Walk (RW)

**Công thức:** $\hat{y}_{t+1} = y_t$

**Metrics:** RMSFE, MAPE

In [ ]:
# Import libraries

In [ ]:
# Load processed data

In [ ]:
# Random Walk model

In [ ]:
# Evaluation

In [ ]:
# Save results